In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils.all_utils import to_categorical

import warnings
warnings.filterwarnings("ignore")

Load the data-set

In [2]:
data = pd.read_csv('icml_face_data.csv')

Make function for preparing the data-set

In [3]:
def prepare_data(data):
    """ Prepare data for modeling
        input: data frame with labels and pixel data
        output: image and label array """
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

Prepare the data-set

In [4]:
test_image_array, test_image_label = prepare_data(data[data[' Usage']=='PublicTest'])

Reshaping the images

In [5]:
test_images = test_image_array.reshape((len(test_image_array), 48, 48, 1))
test_images = test_images.astype('float32')/255

In [6]:
test_labels = to_categorical(test_image_label)

Load thee json model

In [7]:
with open('model.json', 'r') as json_file:
    json_savedModel= json_file.read()
#load the model
model_j = tf.keras.models.model_from_json(json_savedModel)
model_j.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

Load the weights

In [8]:
model_j.load_weights("model_weights.h5")

Compile the model

In [9]:
model_j.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
test_loss, test_acc = model_j.evaluate(test_images, test_labels)

113/113 [==============================] - 5s 13ms/step - loss: 2.0796 - accuracy: 0.1301


In [11]:
predict_images = model_j.predict(test_images)

In [12]:
np.shape(test_image_array)

(3589, 48, 48)

In [13]:
EMOTIONS_LIST = ["Angry", "Disgust",
                 "Fear", "Happy",
                 "Neutral", "Sad",
                 "Surprise"]
# predict the image
EMOTIONS_LIST[np.argmax(predict_images[0])]


'Angry'